In [ ]:
bronze_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/silver/"
gold_path   = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/gold/"
resource_path = "/Volumes/<CATALOG>/<SCHEMA>/bikestore_resource/origem"
resource_path_volume = '/Volumes/aulas_databricks/azure/bikestore_resource/origem/'


In [0]:
%python
silver_map = {
    "tmp_silver_customer":      f"{silver_path}/customer/",
    "tmp_silver_orders":        f"{silver_path}/orders/",
    #"tmp_silver_product":       f"{silver_path}/product/",

}
for view_name, path in silver_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%python
df_orders_pending = spark.sql("""
                               
WITH pending AS(   

SELECT  
  customer_id
  ,order_date
  ,sum(quantity) quantity
  ,store_name
  --,status
  --,lower(status) as minusculo
  --,upper(status) as maiusculo
FROM tmp_silver_orders
WHERE 1=1
AND lower(status) = 'pending' 
--and lower(status) in ('pending','delivered') 
GROUP BY  customer_id,store_name,order_date
 --,status

)
SELECT 
  p.*
  ,c.first_name as first_name_customer
  ,c.email
  ,c.phone

FROM pending  P
LEFT JOIN  tmp_silver_customer c ON P.customer_id = c.customer_id
WHERE c.email IS NOT NULL
AND c.phone IS NOT NULL
         
                              
                              """)

# salvar em Delta na gold
df_orders_pending.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{gold_path}/orders_pending')
    